# python-binance

### 내가 원하는 Symbol의 현재 캔들 데이터 가져오기

In [ ]:
pip install python-binance

In [9]:
import time
import nest_asyncio
nest_asyncio.apply()
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import datetime
import pytz
import pandas as pd

api_key = 'K5zels9GiyObnNIYoesI4DIKC3FxvQLiVY47CBxEaJVPiL0EfpHSnq863hf573Xc'
api_secret = 'NPPcdPyNr9uCxW0PGNuHQp3xRZQFHd3pIHQimJvSQdmi47a1PvR69pGHTnuvNw1Q'
client = Client(api_key, api_secret)

#prices = client.get_all_tickers()
candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1HOUR, limit = 1)

symbol = 'BTCUSDT'


korea_tz = pytz.timezone('Asia/Seoul')
dt_object = datetime.datetime.fromtimestamp(candles[-1][0]/1000, tz=korea_tz)
formatted_datetime = dt_object.strftime("%Y-%m-%d %H:%M")

df = pd.DataFrame(candles, columns=
             [
                'datetime', 
                'open', 
                'high', 
                'low', 
                'close', 
                'volume', 
                'closeTime', 
                'QuoteAssetVolume', 
                'NumTrades',
                'TakerBuyBaseAssetVolume',
                'TakerBuyQuoteAssetVolume'  ,
                 'Ignore'
                
             ]
) 
df['datetime'] = formatted_datetime

df

,datetime,open,high,low,close,volume,closeTime,QuoteAssetVolume,NumTrades,TakerBuyBaseAssetVolume,TakerBuyQuoteAssetVolume,Ignore
0,2024-02-15 05:00,51677.99000000,51795.36000000,51641.10000000,51792.27000000,488.99298000,1707944399999,25289062.90269380,12469,274.54655000,14198486.94162820,0


### 내가 원하는 Symbol의 캔들 데이터 모든 기록 가져오기

In [57]:
klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR,"1 Jan, 2017")
df = pd.DataFrame(klines, columns=
             [
                'datetime', 
                'open', 
                'high', 
                'low', 
                'close', 
                'volume', 
                'closeTime', 
                'QuoteAssetVolume', 
                'NumTrades',
                'TakerBuyBaseAssetVolume',
                'TakerBuyQuoteAssetVolume'  ,
                 'Ignore'
                
             ]
) 
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
df_sorted = df.sort_values(by='datetime')

df_sorted.head()

,datetime,open,high,low,close,volume,closeTime,QuoteAssetVolume,NumTrades,TakerBuyBaseAssetVolume,TakerBuyQuoteAssetVolume,Ignore
0,2017-08-17 13:00:00+09:00,4261.48000000,4313.62000000,4261.32000000,4308.83000000,47.18100900,1502945999999,202366.13839304,171,35.16050300,150952.47794304,0
1,2017-08-17 14:00:00+09:00,4308.83000000,4328.69000000,4291.37000000,4315.32000000,23.23491600,1502949599999,100304.82356749,102,21.44807100,92608.27972836,0
2,2017-08-17 15:00:00+09:00,4330.29000000,4345.45000000,4309.37000000,4324.35000000,7.22969100,1502953199999,31282.31266989,36,4.80286100,20795.31722405,0
3,2017-08-17 16:00:00+09:00,4316.62000000,4349.99000000,4287.41000000,4349.99000000,4.44324900,1502956799999,19241.05829986,25,2.60229200,11291.34701534,0
4,2017-08-17 17:00:00+09:00,4333.32000000,4377.85000000,4333.32000000,4360.69000000,0.97280700,1502960399999,4239.50358563,28,0.81465500,3552.74681715,0


In [56]:
df_sorted.to_csv('BtcUsdt_kline.csv')

### 웹소켓을 활용한 캔들 데이터 실시간 수집

In [12]:
import websocket, json

symbol = 'btcusdt'
interval = '1s'
#window_size = '1h'
url = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'
# url = f'wss://stream.binance.com:9443/ws/{symbol}@ticker_{window_size}'

def convert_timestamp(timestamp):
    korea_tz = pytz.timezone('Asia/Seoul')
    utc_time = datetime.datetime.fromtimestamp(timestamp / 1000, tz=pytz.utc)
    local_time = utc_time.astimezone(korea_tz)
    return local_time.strftime("%Y-%m-%d %H:%M:%S")

def on_message(ws, message):
    data = json.loads(message)
    data = data['k']
    #data['E'] = convert_timestamp(data['E'])
    data['t'] = convert_timestamp(data['t'])
    data['T'] = convert_timestamp(data['T'])
    print(data)
    
def on_close(ws):
    print("close")
    
ws = websocket.WebSocketApp(url, on_message=on_message, on_close=on_close)
ws.run_forever()

{'t': '2024-02-15 05:12:30', 'T': '2024-02-15 05:12:30', 's': 'BTCUSDT', 'i': '1s', 'f': 3417439332, 'L': 3417439364, 'o': '51811.10000000', 'c': '51802.27000000', 'h': '51811.10000000', 'l': '51802.26000000', 'v': '0.50429000', 'n': 33, 'x': True, 'q': '26126.31408540', 'V': '0.05439000', 'Q': '2817.52546530', 'B': '0'}
{'t': '2024-02-15 05:12:31', 'T': '2024-02-15 05:12:31', 's': 'BTCUSDT', 'i': '1s', 'f': 3417439365, 'L': 3417439366, 'o': '51802.27000000', 'c': '51802.26000000', 'h': '51802.27000000', 'l': '51802.26000000', 'v': '0.01358000', 'n': 2, 'x': True, 'q': '703.47482060', 'V': '0.01298000', 'Q': '672.39346460', 'B': '0'}
{'t': '2024-02-15 05:12:32', 'T': '2024-02-15 05:12:32', 's': 'BTCUSDT', 'i': '1s', 'f': 3417439367, 'L': 3417439384, 'o': '51802.26000000', 'c': '51802.27000000', 'h': '51802.27000000', 'l': '51802.26000000', 'v': '0.52166000', 'n': 18, 'x': True, 'q': '27023.17197630', 'V': '0.50247000', 'Q': '26029.08660690', 'B': '0'}
{'t': '2024-02-15 05:12:33', 'T': 

True